# imports

In [3]:
# imports

from config import *
from sys import getsizeof

import json
import pandas as pd
import dateutil.parser as dp


import os
import sys
import pickle
import datetime
import importlib

# datasets

In [4]:
# Loading example dataset

df = pd.read_csv(DATASETS_ABS_PATHS["BWSAS"])
df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True) # Replace spaces with underscores in column names
df.Measurement_Timestamp = pd.to_datetime(df.Measurement_Timestamp, format="%m/%d/%Y %I:%M:%S %p")
df = df.convert_dtypes()
df_dict = df.to_dict(orient="records")
df.dtypes

FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\MANDR\\Desktop\\1210\\gdepc\\static\\datasets\\beach-weather-stations-automated-sensors-1.csv'

In [ ]:
for n, t in df.dtypes.to_dict().items():
    print(f"{n:30} | {t}")

Station_Name                   | string
Measurement_Timestamp          | datetime64[ns]
Air_Temperature                | Float64
Wet_Bulb_Temperature           | Float64
Humidity                       | Int64
Rain_Intensity                 | Float64
Interval_Rain                  | Float64
Total_Rain                     | Float64
Precipitation_Type             | Int64
Wind_Direction                 | Int64
Wind_Speed                     | Float64
Maximum_Wind_Speed             | Float64
Barometric_Pressure            | Float64
Solar_Radiation                | Int64
Heading                        | Int64
Battery_Life                   | Float64
Measurement_Timestamp_Label    | string
Measurement_ID                 | string


In [ ]:
df

,Station_Name,Measurement_Timestamp,Air_Temperature,Wet_Bulb_Temperature,Humidity,Rain_Intensity,Interval_Rain,Total_Rain,Precipitation_Type,Wind_Direction,Wind_Speed,Maximum_Wind_Speed,Barometric_Pressure,Solar_Radiation,Heading,Battery_Life,Measurement_Timestamp_Label,Measurement_ID
0,Oak Street Weather Station,2015-05-22 15:00:00,<NA>,7.0,55,0.0,0.0,1.4,0,63,1.9,2.8,<NA>,780,322,12.0,05/22/2015 3:00 PM,OakStreetWeatherStation201505221500
1,Oak Street Weather Station,2015-05-22 17:00:00,<NA>,6.3,56,0.0,0.0,1.4,0,124,1.5,2.3,<NA>,180,322,12.1,05/22/2015 5:00 PM,OakStreetWeatherStation201505221700
2,Oak Street Weather Station,2015-05-22 18:00:00,<NA>,6.5,54,0.0,0.0,1.4,0,156,1.9,3.4,<NA>,127,322,12.1,05/22/2015 6:00 PM,OakStreetWeatherStation201505221800
3,Oak Street Weather Station,2015-05-22 19:00:00,<NA>,6.3,53,0.0,0.0,1.4,0,150,1.4,4.5,<NA>,67,322,12.1,05/22/2015 7:00 PM,OakStreetWeatherStation201505221900
4,Oak Street Weather Station,2015-05-22 20:00:00,<NA>,6.4,52,0.0,0.0,1.4,0,155,1.1,2.3,<NA>,10,322,12.0,05/22/2015 8:00 PM,OakStreetWeatherStation201505222000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59139,Oak Street Weather Station,2017-11-08 00:00:00,4.7,1.8,60,0.0,0.0,0.0,0,273,0.8,1.5,1005.4,0,2,11.9,11/08/2017 12:00 AM,OakStreetWeatherStation201711082400
59140,Foster Weather Station,2017-11-08 00:00:00,2.89,<NA>,58,<NA>,0.0,<NA>,<NA>,278,2.1,2.3,1004.4,0,<NA>,15.2,11/08/2017 12:00 AM,FosterWeatherStation201711082400
59141,63rd Street Weather Station,2017-11-08 01:00:00,3.4,1.1,66,0.0,0.0,28.8,0,299,1.1,2.2,1004.6,4,354,11.8,11/08/2017 1:00 AM,63rdStreetWeatherStation201711080100
59142,Oak Street Weather Station,2017-11-08 01:00:00,4.2,1.4,59,0.0,0.0,0.0,0,276,0.8,1.7,1005.4,1,2,12.0,11/08/2017 1:00 AM,OakStreetWeatherStation201711080100


# proto files

## creating proto template

In [ ]:
def create_proto_template(package_name: str, df: pd.DataFrame):
    d = []
    for name, dtype in df.dtypes.to_dict().items():
        match dtype:
            case "string":
                d.append(["string", name])
            case "Float64":
                d.append(["float", name])
            case "Int64":
                d.append(["int64", name])
            case "datetime64[ns]":
                d.append(["google.protobuf.Timestamp", name])
    
    template = f'\
syntax = "proto3";\n\
import "google/protobuf/timestamp.proto";\n\n\
package {package_name};\n\
message {package_name}_message {{\n\
'

    template += "\n".join(f"    {e[0]} {e[1]} = {d.index(e) + 1};" for e in d)
    template += "\n}"
    return template

In [ ]:
# Does NOT work (yoinked from net)

import pandas as pd
from google.protobuf.descriptor_pb2 import FieldDescriptorProto
from google.protobuf.descriptor_pb2 import FileDescriptorProto
from google.protobuf.compiler.plugin_pb2 import CodeGeneratorRequest
from google.protobuf.compiler.plugin_pb2 import CodeGeneratorResponse

def create_protobuf_template(df):
    # Create a FileDescriptorProto
    file_descriptor_proto = FileDescriptorProto()

    # Create a message type for the DataFrame
    message_type = file_descriptor_proto.message_type.add()
    message_type.name = "DataFrame"

    # Iterate over the columns of the DataFrame
    for column_name, column_type in df.dtypes.items():
        # Create a field descriptor for each column
        field_descriptor = message_type.field.add()
        field_descriptor.name = column_name

        # Map Pandas dtype to Protobuf field type
        if column_type == "int64":
            field_descriptor.type = FieldDescriptorProto.TYPE_INT64
        elif column_type == "float64":
            field_descriptor.type = FieldDescriptorProto.TYPE_DOUBLE
        elif column_type == "bool":
            field_descriptor.type = FieldDescriptorProto.TYPE_BOOL
        elif column_type == "object":
            field_descriptor.type = FieldDescriptorProto.TYPE_STRING
        elif column_type == "datetime64[ns]":
            field_descriptor.type = FieldDescriptorProto.TYPE_STRING
        else:
            field_descriptor.type = FieldDescriptorProto.TYPE_STRING

    # Return the serialized FileDescriptorProto
    return file_descriptor_proto.SerializeToString()

In [ ]:
# proto template exaple
package_name = "m_beach"

protobuf_template = create_proto_template(package_name, df)
print(protobuf_template)

syntax = "proto3";
import "google/protobuf/timestamp.proto";

package m_beach;
message m_beach_message {
    string Station_Name = 1;
    google.protobuf.Timestamp Measurement_Timestamp = 2;
    float Air_Temperature = 3;
    float Wet_Bulb_Temperature = 4;
    int64 Humidity = 5;
    float Rain_Intensity = 6;
    float Interval_Rain = 7;
    float Total_Rain = 8;
    int64 Precipitation_Type = 9;
    int64 Wind_Direction = 10;
    float Wind_Speed = 11;
    float Maximum_Wind_Speed = 12;
    float Barometric_Pressure = 13;
    int64 Solar_Radiation = 14;
    int64 Heading = 15;
    float Battery_Life = 16;
    string Measurement_Timestamp_Label = 17;
    string Measurement_ID = 18;
}


In [ ]:
# Write the byte string to a file.
proto_filename = package_name + ".proto"

with open(path.join(PROTOS_FOLDER_ABS_PATH, proto_filename), "w") as f:
    f.write(protobuf_template)

## compiling proto file

In [ ]:
# Proto file compilation funciton (creates {proto_filename}_pb2.py file)
def compile_proto(proto_filename):
    proto_abs_path = path.join(PROTOS_FOLDER, proto_filename)
    compile_proto_command = f"{PROTOC_EXE} --python_out=. {proto_abs_path} "

    os.system(compile_proto_command)

In [ ]:
# Proto file compilation exapmle

compile_proto(proto_filename)

## loading data into python descriptor of proto message 

In [ ]:
# util function for timestamps calculaitons, conversion and validation
def str2unix(dt: str) -> tuple[int, int]:
    # Converts ISO8601 timstamp string into POSIX timestamp tuple (seconds, nanoseconds)
    dt_unix = dp.parse(dt).timestamp()
    
    seconds = int(dt_unix)
    nanos   = int(dt_unix % 1 * 1e9)

    return (seconds, nanos)


def pdTimestamp2unix(dt: pd.Timestamp) -> tuple[int, int]:
    return str2unix(dt.isoformat())


def datetime_valid(dt_str):
    try:
        datetime.datetime.fromisoformat(dt_str)
    except:
        return False
    return True


# json encoder and decoder for timestamp conversion handling
class _JSONDecoder(json.JSONDecoder):
    def __init__(self, *args, **kwargs):
        json.JSONDecoder.__init__(
            self, object_hook=self.object_hook, *args, **kwargs)

    def object_hook(self, obj):
        ret = {}
        for key, value in obj.items():
            if key in {'timestamp', 'whatever'}:
                ret[key] = datetime.fromisoformat(value) 
            else:
                ret[key] = value
        return ret

    
class _JSONEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, (datetime.date, datetime.datetime, pd.Timestamp, )):
            return obj.isoformat()
        return json.JSONEncoder.default(obj)

In [ ]:
# Function to fill package_message with data from DataFrame
def fill_package_from_df(package_name: str, df_dict: list[dict], item: int):
    # df_dict - df.to_dict(orient="records"); item - number of record in df
    package = importlib.import_module(PROTOS_FOLDER + "." + package_name + "_pb2")
    package_message = getattr(package, package_name + "_message")()

    for field, value in df_dict[item].items():
        if value is None:
            continue
        if type(value) is pd.Timestamp:
            seconds, nanos = pdTimestamp2unix(value)
            
            setattr(getattr(package_message, field), "seconds", seconds)
            setattr(getattr(package_message, field), "nanos", nanos)
        else:
            try:
                setattr(package_message, field, value)
            except Exception as e:
                print(f"Exception: {e} | field: {field} | value: {value}")
    
    return package_message


In [ ]:
# Function to fill package_message with data from dict
def fill_package_from_dict(package_name: str, message: dict):
    # message - dict (json)
    package = importlib.import_module(PROTOS_FOLDER + "." + package_name + "_pb2")
    package_message = getattr(package, package_name + "_message")()

    for field, value in message.items():
        if value is None:
            continue
        if datetime_valid(value):
            seconds, nanos = str2unix(value)
            setattr(getattr(package_message, field), "seconds", seconds)
            setattr(getattr(package_message, field), "nanos", nanos)
        elif type(value) is pd.Timestamp:
            seconds, nanos = pdTimestamp2unix(value)
            setattr(getattr(package_message, field), "seconds", seconds)
            setattr(getattr(package_message, field), "nanos", nanos)
        else:
            try:
                setattr(package_message, field, value)
            except Exception as e:
                print(f"Exception: {e} | field: {field} | value: {value}")
    
    return package_message

In [ ]:
# example of filling message with data from df and dicts with pd.Timestamp and timestamp in str 
# 
# NOTE: json format does not specify timestamp format, so i used most commonly used - ISO8601, 
# using custom JSONEncoder/JSONDecoder

item = 0
message = df_dict[item] # dict with pd.Timestamp as timestamp
message_json = json.dumps(message, cls=_JSONEncoder)

package_message = fill_package_from_df(package_name, df_dict, item)
package_message1 = fill_package_from_dict(package_name, message)
# message = json.loads(message_json) # dict with str ISO8601 as timestamp
package_message2 = fill_package_from_dict(package_name, json.loads(message_json))

print(package_message == package_message1 == package_message2)

True


In [ ]:
message, message_json

({'Station_Name': 'Oak Street Weather Station',
  'Measurement_Timestamp': Timestamp('2015-05-22 15:00:00'),
  'Air_Temperature': None,
  'Wet_Bulb_Temperature': 7.0,
  'Humidity': 55,
  'Rain_Intensity': 0.0,
  'Interval_Rain': 0.0,
  'Total_Rain': 1.4,
  'Precipitation_Type': 0,
  'Wind_Direction': 63,
  'Wind_Speed': 1.9,
  'Maximum_Wind_Speed': 2.8,
  'Barometric_Pressure': None,
  'Solar_Radiation': 780,
  'Heading': 322,
  'Battery_Life': 12.0,
  'Measurement_Timestamp_Label': '05/22/2015 3:00 PM',
  'Measurement_ID': 'OakStreetWeatherStation201505221500'},
 '{"Station_Name": "Oak Street Weather Station", "Measurement_Timestamp": "2015-05-22T15:00:00", "Air_Temperature": null, "Wet_Bulb_Temperature": 7.0, "Humidity": 55, "Rain_Intensity": 0.0, "Interval_Rain": 0.0, "Total_Rain": 1.4, "Precipitation_Type": 0, "Wind_Direction": 63, "Wind_Speed": 1.9, "Maximum_Wind_Speed": 2.8, "Barometric_Pressure": null, "Solar_Radiation": 780, "Heading": 322, "Battery_Life": 12.0, "Measurement_Ti

## serializing/deserializing + io

In [ ]:
# Serialize/deserialize functions

def proto_serialize(package_message):
    return package_message.SerializeToString()


def proto_deserialize(package_message_serialized, package_name: str):
    package = importlib.import_module(PROTOS_FOLDER + "." + package_name + "_pb2")
    package_class = getattr(package, package_name + "_message")

    package_message = package_class.FromString(package_message_serialized)
    
    return package_message

In [ ]:
# IO funcitons for package_message 

# writing package_message to the file
def write_package(package_message, package_bin_filename):
    with open(package_bin_filename, "wb") as f:
        package_message_serialized = proto_serialize(package_message)
        f.write(package_message_serialized)
    
    return package_message_serialized
    


# reading package_message from the file
def read_package(package_bin_filename, package_name):
    with open(package_bin_filename, "rb") as f:
        package_message_serialized = f.read()
        
        package_message = proto_deserialize(package_message_serialized, package_name)

    return package_message

In [ ]:
# example usage of serialization
package_bin_filename = package_name + ".bin"

package_message_serialized   = write_package(package_message, package_bin_filename)
package_message_deserialized = read_package(package_bin_filename, package_name)


In [ ]:
# dumping json file
package_json_filename = package_name + ".json"
# with open(package_name + ".json", "w") as f:
#     json.dump(message, f, cls=_JSONEncoder)
# with open(package_name + "1.json", "w") as f:
#     f.write(message_json)


In [ ]:
# util funcitons for bit <-> byte conversion and size finding

def to_bits(byte_string: bytes) -> str:
    # Convert byte string into bit string
    return bin(int.from_bytes(byte_string, byteorder='big'))[2:]  # Remove '0b' prefix from binary string


def to_bytes(bit_string: str) -> bytes:
    # Convert bit string into byte string
    return int(bit_string, 2).to_bytes((len(bit_string) + 7) // 8, byteorder='big')


def bitsize(data: bytes | str) -> int:
    if type(data) == bytes:
        return len(to_bits(data))
    elif type(data) == str and set(data) <= {'0', '1'}:
        return len(data)
    else:
        return 0

In [ ]:
# example of using bit/byte conversion + size finding
to_bits(package_message_serialized), bitsize(package_message_serialized), to_bytes(to_bits(package_message_serialized)) == package_message_serialized

('10100001101001001111011000010110101100100000010100110111010001110010011001010110010101110100001000000101011101100101011000010111010001101000011001010111001000100000010100110111010001100001011101000110100101101111011011100001001000000110000010001100000010110100111111001010101000000101001001010000000000000000111000000100000000101000001101110100010100110011001100111011001100111111010100000011111101011101001100110011001111110011001111110110010100110011001100110011001101000000011100001000110000000110011110001100001000000010100001010000000100000000000000000100000001000001100010100000000100010010001100000011010100101111001100100011001000101111001100100011000000110001001101010010000000110011001110100011000000110000001000000101000001001101100100100000000100100011010011110110000101101011010100110111010001110010011001010110010101110100010101110110010101100001011101000110100001100101011100100101001101110100011000010111010001101001011011110110111000110010001100000011000100110101001100000011010100

In [ ]:
# sizes comparison
print(f"message size (dict):                {getsizeof(message)}")
print(f"message_json size (str of json):    {getsizeof(message_json)}")
print()
print(f"bin file size:                      {os.path.getsize(package_bin_filename)}")
print(f"json file size:                     {os.path.getsize(package_json_filename)}")
print()
print(f"package_message size:               {getsizeof(package_message)}")
print(f"package_message_serialized bits:    {bitsize(package_message_serialized)}")
print()
print(f"ratio of serialized/json files:     {os.path.getsize(package_json_filename) / os.path.getsize(package_bin_filename)}")
print(f"package_message == package_message_deserialized: {package_message == package_message_deserialized}")

message size (dict):                464
message_json size (str of json):    563

bin file size:                      131
json file size:                     522

package_message size:               80
package_message_serialized bits:    1044

ratio of serialized/json files:     3.984732824427481
package_message == package_message_deserialized: True


### etc

In [ ]:
print([field.name for field in package_message.DESCRIPTOR.fields])


['Station_Name', 'Measurement_Timestamp', 'Air_Temperature', 'Wet_Bulb_Temperature', 'Humidity', 'Rain_Intensity', 'Interval_Rain', 'Total_Rain', 'Precipitation_Type', 'Wind_Direction', 'Wind_Speed', 'Maximum_Wind_Speed', 'Barometric_Pressure', 'Solar_Radiation', 'Heading', 'Battery_Life', 'Measurement_Timestamp_Label', 'Measurement_ID']


# Compression

## libraries algorithms comparison

In [ ]:
from collections import Counter

import zlib, gzip, bz2, lzma, lz4.frame, zstd, brotli

In [ ]:
pb_msg = package_message_serialized

pb_ratio_zlib     = bitsize(pb_msg) / bitsize(zlib.compress(pb_msg))
pb_ratio_gzip     = bitsize(pb_msg) / bitsize(gzip.compress(pb_msg))
pb_ratio_bz2      = bitsize(pb_msg) / bitsize(bz2.compress(pb_msg))
pb_ratio_lzma     = bitsize(pb_msg) / bitsize(lzma.compress(pb_msg))
pb_ratio_lz4      = bitsize(pb_msg) / bitsize(lz4.frame.compress(pb_msg))
pb_ratio_zstd     = bitsize(pb_msg) / bitsize(zstd.compress(pb_msg, 1))
pb_ratio_brotli   = bitsize(pb_msg) / bitsize(brotli.compress(pb_msg))



print(f'pb_ratio_zlib:        {pb_ratio_zlib}\
      \npb_ratio_gzip:        {pb_ratio_gzip}\
      \npb_ratio_bz2:         {pb_ratio_bz2}\
      \npb_ratio_lzma:        {pb_ratio_lzma}\
      \npb_ration_lz4:        {pb_ratio_lz4}\
      \npb_ration_zstd(1):    {pb_ratio_zstd}\
      \npb_ration_brotli(11): {pb_ratio_brotli}\
'
)

pb_ratio_zlib:        1.079627714581179      
pb_ratio_gzip:        0.9839773798303487      
pb_ratio_bz2:         0.7462473195139385      
pb_ratio_lzma:        0.7092391304347826      
pb_ration_lz4:        0.9038961038961039      
pb_ration_zstd(1):    0.9338103756708408      
pb_ration_brotli(11): 0.999043062200957


In [ ]:
j_msg = bytes(message_json, "utf-8")

j_ratio_zlib      = bitsize(j_msg)  / bitsize(zlib.compress(j_msg))
j_ratio_gzip      = bitsize(j_msg)  / bitsize(gzip.compress(j_msg))
j_ratio_bz2       = bitsize(j_msg)  / bitsize(bz2.compress(j_msg)) 
j_ratio_lzma      = bitsize(j_msg)  / bitsize(lzma.compress(j_msg))
j_ratio_lz4       = bitsize(j_msg)  / bitsize(lz4.frame.compress(j_msg))
j_ratio_zstd      = bitsize(j_msg)  / bitsize(zstd.compress(j_msg, 1))
j_ratio_brotli    = bitsize(j_msg)  / bitsize(brotli.compress(j_msg))

print(f'j_ratio_zlib:         {j_ratio_zlib}\
      \nj_ratio_gzip:         {j_ratio_gzip}\
      \nj_ratio_bz2:          {j_ratio_bz2}\
      \nj_ratio_lzma:         {j_ratio_lzma}\
      \nj_ration_lz4:         {j_ratio_lz4}\
      \nj_ration_zstd(1):     {j_ratio_zstd}\
      \nj_ration_brotli(11):  {j_ratio_brotli}\
'
)

j_ratio_zlib:         1.7061708214139764      
j_ratio_gzip:         1.6430539157811885      
j_ratio_bz2:          1.5220561429092234      
j_ratio_lzma:         1.3733552631578947      
j_ration_lz4:         1.1986792994544933      
j_ration_zstd(1):     1.5451517394522576      
j_ration_brotli(11):  1.928406466512702


## Huffman

In [ ]:
# Huffman encoding algorithm

import heapq
from collections import defaultdict

class Node:
    def __init__(self, char=None, freq=0, left=None, right=None):
        self.char = char
        self.freq = freq
        self.left = left
        self.right = right

    def __lt__(self, other):
        if self.freq == other.freq:
            return self.char < other.char if self.char and other.char else False
        return self.freq < other.freq

    def __eq__(self, other):
        return self.freq == other.freq and self.char == other.char

def build_huffman_tree(freq_dict):
    priority_queue = [Node(char, freq) for char, freq in freq_dict.items()]
    heapq.heapify(priority_queue)

    while len(priority_queue) > 1:
        left = heapq.heappop(priority_queue)
        right = heapq.heappop(priority_queue)
        new_node = Node(freq=left.freq + right.freq, left=left, right=right)
        heapq.heappush(priority_queue, new_node)

    return priority_queue[0]


def build_frequency_dict(data):
    freq_dict = defaultdict(int)
    for char in data:
        freq_dict[char] += 1
    return freq_dict

def build_codewords(node, current_code="", code_dict=None):
    if code_dict is None:
        code_dict = {}

    if node.char is not None:
        code_dict[node.char] = current_code
        return code_dict

    code_dict = build_codewords(node.left, current_code + "0", code_dict)
    code_dict = build_codewords(node.right, current_code + "1", code_dict)

    return code_dict

def huffman_encode(data):
    freq_dict = build_frequency_dict(data)
    huffman_tree = build_huffman_tree(freq_dict)
    codewords = build_codewords(huffman_tree)

    encoded_data = "".join(codewords[char] for char in data)
    return encoded_data, huffman_tree

def huffman_decode(encoded_data, huffman_tree):
    decoded_data = bytearray()
    current_node = huffman_tree

    for bit in encoded_data:
        if bit == "0":
            current_node = current_node.left
        else:
            current_node = current_node.right

        if current_node.char is not None:
            decoded_data.append(ord(current_node.char))
            current_node = huffman_tree

    return bytes(decoded_data)


def serialize_tree(node):
    if node.char is not None:
        return '1' + bin(node.char)[2:].zfill(8)
    else:
        return '0' + serialize_tree(node.left) + serialize_tree(node.right)


def deserialize_tree(data):
    def helper(index):
        if data[index] == '1':
            char = chr(int(data[index + 1:index + 9], 2))
            return Node(char=char), index + 9
        else:
            left, index = helper(index + 1)
            right, index = helper(index)
            return Node(left=left, right=right), index

    root, _ = helper(0)
    return root


In [ ]:
# example of huffman usage

encoded_data, huffman_tree = huffman_encode(package_message_serialized)

# Serialize the Huffman tree
serialized_tree = serialize_tree(huffman_tree)

# Deserialize the Huffman tree
deserialized_tree = deserialize_tree(serialized_tree)

# The rest of the code remains the same...
# encoded_data = "".join(build_codewords(deserialized_tree)[char] for char in package_message_serialized)
decoded_data = huffman_decode(encoded_data, deserialized_tree)

print(f"Original data:      {package_message_serialized}")
print(f"Decoded data:       {decoded_data}")
print(f"Decoded data == Original data: {package_message_serialized == decoded_data}")
print(f"Encoded data:       {encoded_data}")
print(f"Serialized tree:    {serialized_tree}")
print()
print(f"Encoded data bit length:    {bitsize(encoded_data)}")
print(f"Serialized tree bit length: {bitsize(serialized_tree)}")
print(f"Compression ratio:          {(bitsize(encoded_data) + bitsize(serialized_tree)) / bitsize(package_message_serialized)}")

Original data:      b'\n\x1aOak Street Weather Station\x12\x06\x08\xc0\xb4\xfc\xaa\x05%\x00\x00\xe0@(7E33\xb3?P?]33\xf3?e333@p\x8c\x06x\xc2\x02\x85\x01\x00\x00@A\x8a\x01\x1205/22/2015 3:00 PM\x92\x01#OakStreetWeatherStation201505221500'
Decoded data:       b'\n\x1aOak Street Weather Station\x12\x06\x08\xc0\xb4\xfc\xaa\x05%\x00\x00\xe0@(7E33\xb3?P?]33\xf3?e333@p\x8c\x06x\xc2\x02\x85\x01\x00\x00@A\x8a\x01\x1205/22/2015 3:00 PM\x92\x01#OakStreetWeatherStation201505221500'
Decoded data == Original data: True
Encoded data:       10011011011010001011000101111011010100001110100011100110011101101001101011000001111001110011001000111010100001110000111100110111001011011001010111010101001100010001101000101011111011111010100011001000010010100110100110010010010011011100011001100110011011111111111111110011111101000010011001110111101111111100001100110011001000111010011000010101001110111010010101000010011101111010100101001001000110010100111111110110101101011101100101000000000001010000001011111101101111

# rANS PyComP

In [ ]:
from collections import Counter
counter = Counter(pb_msg)

pb_msg_chr = ''.join([chr(i) for i in counter.keys()])

In [ ]:
from libs.PyComP import ANS
msg_counter = Counter(pb_msg_chr)
ans = ANS.rANS(list(msg_counter.keys()), list(msg_counter.values()))
msg_enc, final_state = ans.encode([chr(i) for i in pb_msg], 0)
msg_dec = ''.join(ans.decode(msg_enc, final_state))


In [ ]:
pb_msg_chr, pb_msg_chr, msg_enc[2:], len(msg_enc[2:])

('\n\x1aOak StreWhion\x12\x06\x08À´üª\x05%\x00à@(7E3³?P]óp\x8cxÂ\x02\x85\x01A\x8a05/21:M\x92#',
 '\n\x1aOak StreWhion\x12\x06\x08À´üª\x05%\x00à@(7E3³?P]óp\x8cxÂ\x02\x85\x01A\x8a05/21:M\x92#',
 '10101100111110011010101001110001101010001010010100010101000101110111101000011001001100101110001100010110001011011111100110000101111010001101101000110101010001111000100011101110010010111101100111111101111011010000100110001001111101111010110100001111000110010010101100010111100000111000100011000001001111110000001110001101001100101101111110110110101110111100101111100000000111010101010111101011010101011011011001011000100110010110011100111010000101101010000100001000011000110111000100010111101001011110000101000010011011110010111011000110111111111111001111110001100011000101100100001100110100111110111010011111101100101101101111010011011111110001011001101110111011101101000010011110100100110011010100111100100011110001110011101011000000100111011',
 743)

# Cooking

In [ ]:
# df_paths = DATASETS_ABS_PATHS.values()
# dfs = 
# df = pd.read_csv(DATASETS_ABS_PATHS["BWSAS"])
# df.rename(columns=lambda x: x.replace(' ', '_'), inplace=True) # Replace spaces with underscores in column names
# df.Measurement_Timestamp = pd.to_datetime(df.Measurement_Timestamp, format="%m/%d/%Y %I:%M:%S %p")
# df = df.convert_dtypes()
# df.dtypes

In [ ]:
DATASETS_REL_PATHS

{'ASCCDCV': 'static\\datasets\\all_sites_combined_concentration_data_clean_version.xlsx',
 'ARGAZAL': 'static\\datasets\\argentine_all_zones_all_winds.xlsx',
 'ARMAZAL': 'static\\datasets\\armourdale_all_zones_all_winds.xlsx',
 'BWQAS': 'static\\datasets\\beach_water_quality_automated_sensors_1.csv',
 'BWSAS': 'static\\datasets\\beach_weather_stations_automated_sensors_1.csv',
 'DTFSHZPG': 'static\\datasets\\data_table_for_science_hub_zhou_paper_gullett.xlsx',
 'IOTNL': 'static\\datasets\\iot_network_logs.csv',
 'IOTTEMP': 'static\\datasets\\iot_temp.csv',
 'IOT1': 'static\\datasets\\iotpond1.csv',
 'IOT2': 'static\\datasets\\iotpond2.csv',
 'IOT3': 'static\\datasets\\iotpond3.csv',
 'IOT4': 'static\\datasets\\iotpond4.csv',
 'IOT6': 'static\\datasets\\iotpond6.csv',
 'IOT7': 'static\\datasets\\iotpond7.csv',
 'IOT8': 'static\\datasets\\iotpond8.csv',
 'IOT9': 'static\\datasets\\iotpond9.csv',
 'IOT10': 'static\\datasets\\iotpond10.csv',
 'IOT11': 'static\\datasets\\iotpond11.csv',
 'I

In [ ]:
df_paths = list(DATASETS_REL_PATHS.values())

In [ ]:
df_paths

['static\\datasets\\all_sites_combined_concentration_data_clean_version.xlsx',
 'static\\datasets\\argentine_all_zones_all_winds.xlsx',
 'static\\datasets\\armourdale_all_zones_all_winds.xlsx',
 'static\\datasets\\beach_water_quality_automated_sensors_1.csv',
 'static\\datasets\\beach_weather_stations_automated_sensors_1.csv',
 'static\\datasets\\data_table_for_science_hub_zhou_paper_gullett.xlsx',
 'static\\datasets\\iot_network_logs.csv',
 'static\\datasets\\iot_temp.csv',
 'static\\datasets\\iotpond1.csv',
 'static\\datasets\\iotpond2.csv',
 'static\\datasets\\iotpond3.csv',
 'static\\datasets\\iotpond4.csv',
 'static\\datasets\\iotpond6.csv',
 'static\\datasets\\iotpond7.csv',
 'static\\datasets\\iotpond8.csv',
 'static\\datasets\\iotpond9.csv',
 'static\\datasets\\iotpond10.csv',
 'static\\datasets\\iotpond11.csv',
 'static\\datasets\\iotpond12.csv',
 'static\\datasets\\turner_all_zones_all_winds.xls']

In [ ]:
pd.read_csv("static\\datasets\\iot_temp.csv")

,id,room_id/id,noted_date,temp,out/in
0,__export__.temp_log_196134_bd201015,Room Admin,08-12-2018 09:30,29,In
1,__export__.temp_log_196131_7bca51bc,Room Admin,08-12-2018 09:30,29,In
2,__export__.temp_log_196127_522915e3,Room Admin,08-12-2018 09:29,41,Out
3,__export__.temp_log_196128_be0919cf,Room Admin,08-12-2018 09:29,41,Out
4,__export__.temp_log_196126_d30b72fb,Room Admin,08-12-2018 09:29,31,In
...,...,...,...,...,...
97601,__export__.temp_log_91076_7fbd08ca,Room Admin,28-07-2018 07:07,31,In
97602,__export__.temp_log_147733_62c03f31,Room Admin,28-07-2018 07:07,31,In
97603,__export__.temp_log_100386_84093a68,Room Admin,28-07-2018 07:06,31,In
97604,__export__.temp_log_123297_4d8e690b,Room Admin,28-07-2018 07:06,31,In


In [ ]:
# dfs = [pd.read_csv(df_path) for df_path in df_paths]
dfs = []
for df_path in df_paths:
    print(df_path)
    dfs.append(pd.read_csv(df_path, encoding="utf-8"))

static\datasets\all_sites_combined_concentration_data_clean_version.xlsx


ParserError: Error tokenizing data. C error: Expected 2 fields in line 5, saw 3
